In [ ]:
! pip3 install openai transformers plotly matplotlib scikit-learn torch torchvision scipy tiktoken

In [5]:
import pandas as pd
import tiktoken
import openai
from openai import OpenAI


In [3]:
# Define your embedding model details
embedding_model = "text-embedding-3-large"
max_tokens=8192

In [6]:
client = OpenAI(api_key='', max_retries=5)

# initialize tokenizer
encoding = tiktoken.get_encoding("cl100k_base")

def get_embedding(text: str, model=embedding_model, **kwargs) -> List[float]:
    # replace newlines
    text = text.replace("\n", " ")

    # truncate tokens
    tokens = encoding.encode(text)

    if len(tokens) > max_tokens:
        text = encoding.decode(tokens[:max_tokens])

    response = client.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding


In [39]:
# Read data
df = pd.read_csv("/content/data_frc_1791_norm_text.csv")

In [40]:
df.head()

,Unnamed: 0.1,Unnamed: 0,subject,search_id,dfate,physical-description,rerepublish,volume,text_content,creator,...,coverage,foldout-operator,camera,updatedate,publicdate,citation,addeddate,link-to-catalog,updater,source
0,0,0,"Public, Finance",NaN,NaN,NaN,NaN,NaN,va de dépense qve rend le directoire du départ...,Ardèche (France). Directoire.,...,France--Ardèche,NaN,NaN,NaN,2016-05-10 21:20:59,NaN,2016-05-10 21:20:59,NaN,NaN,NaN
1,1,8,"Mirabeau, Honoré-Gabriel de Riquetti, comte d...",NaN,NaN,NaN,NaN,NaN,éloge funebre de mirabeau prononcé le mai la c...,Société des amis de la constitution (Sèvres...,...,NaN,NaN,Canon EOS 5D Mark II,2016-08-17 15:12:42,2016-08-17 13:56:08,NaN,2016-08-17 15:12:44,NaN,chrissy-robinson@archive.org,NaN
2,2,24,Church and state,NaN,1791.0,NaN,NaN,NaN,nouvelle instruction en forme de conférence ou...,"Pichard, Jean-Louis, 1759-1819, publisher.",...,NaN,NaN,NaN,2017-04-19 13:15:32,2017-04-19 12:13:44,"Barbier, A.A. Ouvrages anonymes, III, column...",2017-04-19 13:15:34,NaN,chrissy-robinson@archive.org,NaN
3,3,36,NaN,NaN,NaN,NaN,NaN,NaN,triomphe de rité un ay san the ri yertissement...,NaN,...,NaN,NaN,Canon EOS 5D Mark II,2017-01-10 18:50:26,2017-01-10 17:29:00,NaN,2017-01-10 18:50:28,NaN,chrissy-robinson@archive.org,NaN
4,4,52,France. Assemblée nationale constituante (178...,NaN,NaN,NaN,NaN,NaN,compte rendu ses commettans par de bonneville ...,France. États généraux.,...,NaN,NaN,Canon EOS 5D Mark II,2016-08-15 18:17:02,2016-08-15 17:10:02,"Martin & Walter. Révolution française, I, ...",2016-08-15 18:17:05,NaN,chrissy-robinson@archive.org,NaN


In [41]:
non_string_entries = df[df['text_content'].apply(lambda x: not isinstance(x, str))]

# Print the indices and values of non-string entries
print("Non-string entries found at indices:")
print(non_string_entries)

Non-string entries found at indices:
Empty DataFrame
Columns: [Unnamed: 0.1, Unnamed: 0, subject, search_id, dfate, physical-description, rerepublish, volume, text_content, creator, sponsor, ocr, language, bwocr, repub_state, languate, page-progression, scanfee, file_name, contributor, year, invoice, link_to_catalog, foldoutcount, scanner, uploader, republisher_date, collection, physical_description, curation, item_number, scanningcenter, openlibrary_edition, shiptracking, scandate, openlibrary_work, sponsordate, date, repub_seconds, identifier, identifier-ark, mediatype, notes, identifier-access, topicmodel, publisher, republisher_operator, republisher_time, operator, ppi, description, bookplateleaf, republisher, title, imagecount, allsubjects, foldout_seconds, bookreader-defaults, lccn, backup_location, call_number, coverage, foldout-operator, camera, updatedate, publicdate, citation, addeddate, link-to-catalog, updater, source]
Index: []

[0 rows x 71 columns]


In [33]:
# Remove rows where 'text_content' is NaN
df = df.dropna(subset=['text_content'])

df.reset_index(drop=True, inplace=True)


In [42]:
len(df)

1320

In [43]:
df['openai_embedding'] = df['text_content'].apply(get_embedding)

In [44]:
df['openai_embedding'].head()

0    [0.035654012113809586, -0.011971485801041126, ...
1    [0.018895348533988, -0.03015376441180706, -0.0...
2    [0.03520096838474274, -0.046211861073970795, -...
3    [0.02609384059906006, -0.036538705229759216, -...
4    [0.03917008265852928, -0.031102433800697327, 0...
Name: openai_embedding, dtype: object

In [45]:
# size of each embedding
enc_1 = df['openai_embedding'].iloc[100]
len(enc_1)

3072

In [46]:
df.to_csv('data_frc_1791_norm_text_with_embedding.csv')